# Step 2: Data Wrangling

The collected data will be prepped for model building here. In general:
- Blanks will be filled with either the average (numerical) or FALSE (boolean)
- Duplicate samples will be removed
- Data will be converted to consistent units
- Features will be engineered (outliers, etc.)
- Training data (recentlySold) will be modified en masse to have a matching statistical distribution as 'live' data (forSale)

There are two primary steps in this process:
1) Narrow down feature set from 600+ features (manual selection)
2) Clean, organize, scale, and engineer data for use in model building

---

In [93]:
'''
Import libraries
'''

import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ks_2samp, zscore

pd.options.display.max_columns = None

In [94]:
'''
Define Zillow data parameters
'''

# IMPORTANT: 'priceRange' here MuST match the 'priceRange' found in the data collection step.
priceRange = [
    [1, 250000], [250001, 300000], [300001, 350000], [350001, 400000],
    [400001, 450000], [450001, 500000], [500001, 750000], [750001, 1000000],
    [1000001, 2000000], [2000001, 5000000], [5000001, 50000000]
    ]

outlierBoolean_fs = []
outlierBoolean_rs = []

In [95]:
'''
Data Distribution Variables
'''

removeOutliers = False # Remove outliers before scaling and exporting to CSV?
scale = False # Scale data using ECDF before exporting to CSV? <-- This screws up everything. LEAVE SET TO FALSE.
OHE_on = False # Turn on or off the one-hot-encoded variables.

In [96]:
'''
Build helper functions for code cleanliness
'''

def hoaFees(series: pd.DataFrame):
    '''
    Standardizes the units for HOA fees to monthly.

    ## Parameters:
    - series: Input data as pd.Series.

    ## Returns:
    - payment: Output data as pd.Series.
    '''
    paymentFreq = series.apply(lambda x: str(x).strip("$,1234567890 "))
    payment = series.apply(lambda x: str(x).strip("$,-abcdefghijklmnopqrstuvwxyz ").replace(',', ''))
    payment = payment.apply(lambda x: float(x))
    paymentFreq.replace('', 0, inplace=True)
    for i, freq in enumerate(paymentFreq):
        if freq == 'monthly':
            payment[i] = payment[i]
        elif freq == 'quarterly':
            payment[i] = payment[i]/4
        elif freq == 'semi-annually':
            payment[i] = payment[i]/6
        elif freq == 'annually':
            payment[i] = payment[i]/12
        else:
            payment[i] = 0    
    payment = round(payment)
    return payment


# Concat partial URL with website prefix
def fullURL(hdpURL: str):
    '''
    Provides a complete URL for each home based on their partial URL.

    ## Parameters:
    - hpdURL: Input string.

    ## Returns:
    - hdpURL: Output string.
    '''
    hdpURL = 'www.zillow.com' + str(hdpURL)
    return hdpURL


# Set area units to a consistent value of 'Square Feet'
def areaUnitConversion(units: pd.DataFrame):
    '''
    Standardizes the units for lot and home area to square feet.

    ## Parameters:
    - units: Input data as dataframe containing lot area value and units.

    ## Returns:
    - units_copy: Output as dataframe containing the adjusted values.
    '''
    # Input validation
    if 'lotAreaUnits' not in units.columns or 'lotAreaValue' not in units.columns:
        raise ValueError("Input dataframe does not have the correct columns present.")
    # Create a copy of the DataFrame to avoid in-place changes
    units_copy = units.copy(deep=True)
    for idx in units_copy.index:
        if units_copy.loc[idx, 'lotAreaUnits'] == 'Acres':
            units_copy.loc[idx, 'lotAreaValue'] *= 43560
            units_copy.loc[idx, 'lotAreaUnits'] = 'Square Feet'
        elif units_copy.loc[idx, 'lotAreaUnits'] == 'sqft':
            units_copy.loc[idx, 'lotAreaUnits'] = 'Square Feet'
    return units_copy


def timeOnZillowConversion(toz: str):
    '''
    Standardizes the units for time on Zillow.

    ## Parameters:
    - toz: Input string for time on Zillow.

    ## Returns:
    - units: Output string for adjusted units.
    '''
    units = toz.strip(' 1234567890')
    if units == 'day':
        units = toz
    elif units == 'days':
        units = toz
    else:
        units = '1 day'
    return units


'''
Function to calculate outliers and return list of outliers and non-outliers.
'''
def outliersAll_IQR(sortedList: pd.DataFrame, outlierBoolean: list, outlierColumn: str):
    if any(outlierBoolean)==False:
        outlierBoolean = [0] * sortedList.shape[0]
    
    itemIndex = sortedList[outlierColumn].index
    Q1 = np.percentile(sortedList[outlierColumn], 25)
    Q3 = np.percentile(sortedList[outlierColumn], 75)
    IQR = Q3 - Q1
    upperBound = Q3 + 1.5*IQR
    lowerBound = Q1 - 1.5*IQR
    for i, value in enumerate(sortedList[outlierColumn]):
        if (value < lowerBound or value > upperBound):
            outlierBoolean[itemIndex[i]] = 1
    return np.array(outlierBoolean)


def outliersAll_Z_Score(sortedList: pd.DataFrame, outlierBoolean: list, outlierColumn: str, bound: float=[-3,3]):
    if any(outlierBoolean)==False:
        outlierBoolean = [0] * sortedList.shape[0]

    itemIndex = sortedList[outlierColumn].index
    score = zscore(np.array(sortedList[outlierColumn]))
    for i, value in enumerate(score):
        if value < bound[0] or value > bound[1]:
            outlierBoolean[itemIndex[i]] = 1
    return np.array(outlierBoolean)


def outliers_IQR(sortedList: pd.DataFrame, outlierBoolean: list, outlierColumn: str, groupbyColumn: str):
    '''
    Identify outliers in column data when grouped by another column.

    ## Parameters:
    - sortedList: Input dataframe
    - outlierBoolean: List of current ouliers from previous calls. If empty, will be initialized.
    - outlierColumn: Column name for outlier detection.
    - groupbyColumn: Column name for outliers to be grouped by.

    ## Returns:
    - outlierBoolean: pd.Series of 1(True)/0(False) based on outlierColumn and groupbyColumn selected.
    '''
    if any(outlierBoolean)==False:
        outlierBoolean = [0] * sortedList.shape[0]

    for item in sortedList[groupbyColumn].unique():
        itemIndex = sortedList.index[sortedList[groupbyColumn]==item]
        Q1 = np.percentile(sortedList[outlierColumn].loc[sortedList[groupbyColumn]==item], 25)
        Q3 = np.percentile(sortedList[outlierColumn].loc[sortedList[groupbyColumn]==item], 75)
        IQR = Q3 - Q1
        upperBound = Q3 + 1.5*IQR
        lowerBound = Q1 - 1.5*IQR
        for TF in itemIndex:
            if (sortedList[outlierColumn][TF] < lowerBound or sortedList[outlierColumn][TF] > upperBound):
                outlierBoolean[TF] = 1
    return np.array(outlierBoolean)


def outliers_Z_Score(sortedList: pd.DataFrame, outlierBoolean: list, outlierColumn: str, groupbyColumn: str, bound: float=[-3,3]):
    '''
    Identify outliers in column data when grouped by another column. Outlier detection uses Z-Score.
    Setting upper and lower bounds by 'n' standard deviations from mean where |n| > 0. Does not have to be symmetric.

    ## Parameters:
    - sortedList: Input dataframe
    - outlierBoolean: List of current ouliers from previous calls. If empty, will be initialized.
    - outlierColumn: Column name for outlier detection.
    - groupbyColumn: Column name for outliers to be grouped by.
    - bound: 1x2 array containing upper and lower bounds. bound=[lwr, upr].

    ## Returns:
    - outlierBoolean: pd.Series of 1(True)/0(False) based on outlierColumn and groupbyColumn selected.
    '''
    if any(outlierBoolean)==False:
        outlierBoolean = [0] * sortedList.shape[0]

    for item in sortedList[groupbyColumn].unique():
        itemIndex = sortedList.index[sortedList[groupbyColumn]==item]
        score = zscore(np.array(sortedList[outlierColumn].loc[sortedList[groupbyColumn]==item]))
        for i, TF in enumerate(itemIndex):
            if score[i] < bound[0] or score[i] > bound[1]:
                outlierBoolean[TF] = 1
    return np.array(outlierBoolean)

'''
Function to take any boolean feature and deal with blanks
'''
def cleanBoolean(feature: pd.Series, replaceBlank=False):
    '''
    Replaces all blank and NaN values in a boolean feature with either True or False.

    ## Parameters:
    - feature: Input feature to be cleaned of all blanks and NaN values.
    - replaceBlank: What to replace blanks and Nan's with. Default = False.

    ## Returns:
    - feature: Cleaned feature column.
    '''
    feature = feature.replace(r'\s+', replaceBlank, regex=True)
    feature = feature.replace(np.nan, replaceBlank)
    return feature

def cleanValue(feature: pd.Series, replaceBlank: dict):
    '''
    Replaces all blank and NaN values in a numeric feature with specified value.

    ## Parameters:
    - feature: Input feature to be cleaned of all blanks and NaN values.
    - replaceBlank: Dictionary containing replacement value.

    ## Returns:
    - feature: Cleaned feature column.
    '''

    if 'replace_value' not in replaceBlank:
        raise ValueError("Input 'replaceBlank' argument does not contain 'replace_value'.")
    feature = feature.fillna(value=replaceBlank.get('replace_value'))
    feature.replace(r'\s+', replaceBlank.get('replace_value'), regex=True, inplace=True)
    return feature

'''
Parse dictionary format for school rankings and return highest ranking and shortest distance (not necessarily case-consistent)
'''
def schoolScores(schools: pd.Series):
    finalRating = []
    finalDistance = []
    for i in range(0, schools.shape[0]):
        distance = []
        grades = []
        rating = []
        level = []
        item = eval(schools.iloc[i])
        for j in range(0, len(item)):
            distance.append(item[j].get('distance'))
            grades.append(item[j].get('grades'))
            rating.append(item[j].get('rating'))
            level.append(item[j].get('level'))
        schoolDict = {'distance':distance, 'grades':grades, 'rating':rating, 'level':level}
        df_school = pd.DataFrame(schoolDict)
        df_school = df_school[['distance', 'rating']].groupby(df_school['level']).max().reset_index()
        finalRating.append(df_school['rating'].max())
        finalDistance.append(df_school['distance'].min())
    output = pd.DataFrame({'schoolMaxRating':finalRating, 'schoolMinDistance':finalDistance})
    return output


class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    # def fit(self,X,y=None):
    #     return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        # return self.fit(X,y).transform(X)
        return self.transform(X)
    

def removeDissimilarCols(df_A: pd.DataFrame, df_B: pd.DataFrame, id):
    '''
    Remove all columns and corresponding rows with id from df_A based on columns in df_B. 
    Only use this when the column differences between df_A and df_B are similarly encoded.
    This will, unintentionally remove an entire column if not carefull.
    
    ## Parameters:
    - df_A: Dataframe from which to remove columns. Will also remove any rows containing value=id from the columns to be removed.
    - df_B: Dataframe used to determine which columns are ONLY present in df_A.
    - id: Value used to identify which rows to remove from df_A.

    ## Output:
    - df_A: This new dataframe will not contain any columns not present in df_B and will have had any corresponding rows removed.
    '''
    col_to_remove = set(df_A.columns) - set(df_B.columns)
    for col in col_to_remove:
        df_A.drop(df_A[df_A[col] == id].index, inplace=True) # remove rows 
        df_A.drop(columns=col, inplace=True)
    return df_A


def match_ecdf(series, reference):
    '''
    Match one feature's distribution to another's using Empirical Cumulative Density Function
    
    ## Parameters:
    - series: 
    - reference: 

    ## Output:
    - matched_series: 
    '''
    sorted_series = np.sort(series, kind='mergesort')
    sorted_reference = np.sort(reference, kind='mergesort')
    ecdf_series = np.linspace(0, 1, len(series), endpoint=False)
    ecdf_reference = np.linspace(0, 1, len(reference), endpoint=False)
    matched_values = np.interp(ecdf_series, ecdf_reference, sorted_reference)
    matched_series = np.interp(sorted_series, sorted_series, matched_values)
    return matched_series

In [97]:
'''
Read in all the housing data from data collection and populate initial data frame.
'''

df_forSale = pd.DataFrame()
df_recentlySold = pd.DataFrame()

for i, price in enumerate(priceRange):
    df_forSale = pd.concat([df_forSale, pd.read_csv(f'Raw_Housing_Data/housingData_{price[0]}_{price[1]}.csv',index_col=0)],
                           axis=0, ignore_index=True)
    df_recentlySold = pd.concat([df_recentlySold, pd.read_csv(f'Raw_Housing_Data/housingData_recentlySold_{price[0]}_{price[1]}.csv',index_col=0)],
                                axis=0, ignore_index=True)

In [98]:
'''
Create file with dataframe column names. Definitely not required, but does make the next step a bit easier when dealing with a smaller file.
'''

columnNames_forSale = list(map(lambda x: x.split(', '), df_forSale.columns))
columnNames_recentlySold = list(map(lambda x: x.split(','), df_recentlySold))

with open('Raw_Housing_Data/dataframeColumnNames_forSale.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_forSale)

with open('Raw_Housing_Data/dataframeColumnNames_recentlySold.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_recentlySold)

In [99]:
'''
Manually downselect column names to keep for future consideration.
'''

columnNames = ['zpid','timeOnZillow.1','zipcode','yearBuilt','bedrooms','bathrooms','livingAreaValue','livingAreaUnits',
                       'lotAreaValue','lotAreaUnits','zestimate','latitude','longitude','resoFacts.isSeniorCommunity','resoFacts.propertySubType','hdpUrl']

addtl = ['cityId','schools','resoFacts.garageSpaces','resoFacts.hasWaterfrontView','resoFacts.taxAnnualAmount']

columnNames_forSale = columnNames + addtl + ['price']
columnNames_recentlySold = ['dateSoldString'] + columnNames_forSale

df_forSale_trunc = df_forSale.loc[:,columnNames_forSale]
df_recentlySold_trunc = df_recentlySold.loc[:,columnNames_recentlySold]

del df_forSale, df_recentlySold

In [100]:
'''
Clean object features.

Set True = 1, False = 0.
'''

# Set any value less than 1 day to be equal to 1 day. Set to Int.
# Drop all labels with lotAreaValue = NaN
df_forSale_trunc.dropna(subset=['timeOnZillow.1'], inplace=True)
df_recentlySold_trunc.dropna(subset=['timeOnZillow.1'], inplace=True)
df_recentlySold_trunc['timeOnZillow.1'] = df_recentlySold_trunc['timeOnZillow.1'].apply(timeOnZillowConversion)
df_recentlySold_trunc['timeOnZillow.1'] = df_recentlySold_trunc['timeOnZillow.1'].apply(lambda x: x.strip(' days')).astype(int)
df_recentlySold_trunc.rename(columns={'timeOnZillow.1':'daysOnZillow'}, inplace=True)
df_forSale_trunc['timeOnZillow.1'] = df_forSale_trunc['timeOnZillow.1'].apply(timeOnZillowConversion)
df_forSale_trunc['timeOnZillow.1'] = df_forSale_trunc['timeOnZillow.1'].apply(lambda x: x.strip(' days')).astype(int)
df_forSale_trunc.rename(columns={'timeOnZillow.1':'daysOnZillow'}, inplace=True)

# Need to keep lotAreaUnits and livingAreaUnits consistently set to 'Square Feet'.
# livingAreaUnits already only contains 'Square Feet' or NaN.
df_forSale_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(df_forSale_trunc[['lotAreaValue','lotAreaUnits']])
df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']])

# Set NaN values to 'False'. True is already designated by default, where applicable.
df_forSale_trunc['resoFacts.isSeniorCommunity'] = cleanBoolean(df_forSale_trunc['resoFacts.isSeniorCommunity'].copy())
df_recentlySold_trunc['resoFacts.isSeniorCommunity'] = cleanBoolean(df_recentlySold_trunc['resoFacts.isSeniorCommunity'].copy())

# Reformat the string representing the property subtype so it can be easily worked on.
# Removing any RV Community data as it is not a valid 'single family home' sub type for this scope.
df_forSale_trunc['resoFacts.propertySubType'] = df_forSale_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_recentlySold_trunc['resoFacts.propertySubType'] = df_recentlySold_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['resoFacts.propertySubType']!='RV Community']
df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['resoFacts.propertySubType']!='RV Community']
df_forSale_trunc.drop(columns=['resoFacts.propertySubType'], inplace=True)
df_recentlySold_trunc.drop(columns=['resoFacts.propertySubType'], inplace=True)

# Add prefix to website
df_forSale_trunc['hdpUrl']=df_forSale_trunc['hdpUrl'].apply(fullURL)
df_recentlySold_trunc['hdpUrl']=df_recentlySold_trunc['hdpUrl'].apply(fullURL)

df_forSale_trunc.dropna(subset=['zipcode', 'cityId'], inplace=True)
df_recentlySold_trunc.dropna(subset=['zipcode', 'cityId'], inplace=True)

# Parse dictionary and pickout highest rated school score and shortest school distance
df_forSale_trunc = pd.concat([df_forSale_trunc.copy(), schoolScores(df_forSale_trunc['schools'])], axis=1, ignore_index=False)
df_forSale_trunc.drop(columns=['schools'], inplace=True)
df_recentlySold_trunc = pd.concat([df_recentlySold_trunc, schoolScores(df_recentlySold_trunc['schools'])], axis=1, ignore_index=False)
df_recentlySold_trunc.drop(columns=['schools'], inplace=True)

df_forSale_trunc.dropna(subset=['schoolMaxRating', 'schoolMinDistance'], inplace=True)
df_recentlySold_trunc.dropna(subset=['schoolMaxRating', 'schoolMinDistance'], inplace=True)

# Clean boolean variable
df_forSale_trunc['resoFacts.hasWaterfrontView'] = cleanBoolean(df_forSale_trunc['resoFacts.hasWaterfrontView'].copy())
df_recentlySold_trunc['resoFacts.hasWaterfrontView'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasWaterfrontView'].copy())

# Convert dateSoldString to datetime object
df_recentlySold_trunc['dateSoldString'] = pd.to_datetime(df_recentlySold_trunc['dateSoldString'])

In [101]:
'''
Clean numerical features.

Set all numerical values to float32.
'''

# Set zipcode to string value
df_forSale_trunc['zipcode'] = df_forSale_trunc['zipcode'].astype(str)
df_recentlySold_trunc['zipcode'] = df_recentlySold_trunc['zipcode'].astype(str)

# Convert yearBuilt to integer and drop any rows that have yearBuilt as NaN
df_forSale_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_recentlySold_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_forSale_trunc['yearBuilt']=df_forSale_trunc['yearBuilt'].astype(int)
df_recentlySold_trunc['yearBuilt']=df_recentlySold_trunc['yearBuilt'].astype(int)

# Replace bedroom and bathroom blanks with 1
# Drop all bathroom and bedroom values of NaN
df_forSale_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)
df_recentlySold_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)
value = {'replace_value':1}
df_forSale_trunc['bedrooms'] = cleanValue(df_forSale_trunc['bedrooms'].copy(), value)
df_recentlySold_trunc['bedrooms'] = cleanValue(df_recentlySold_trunc['bedrooms'].copy(), value)
df_forSale_trunc['bathrooms'] = cleanValue(df_forSale_trunc['bathrooms'].copy(), value)
df_recentlySold_trunc['bathrooms'] = cleanValue(df_recentlySold_trunc['bathrooms'].copy(), value)

# Drop all labels with livingAreaValue = NaN -- This is the primary feature for price prediction.
# Could try to do more searches on $/sqft but for now this will suffice.
df_forSale_trunc.dropna(subset=['livingAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['livingAreaValue'], inplace=True)
# Drop all livingAreaValue rows <= 500. Zero here will cause inf. on price/sqft calc. <- breaks feature scaling and model building.
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['livingAreaValue']>=500]
df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['livingAreaValue']>=500]

# Drop all labels with lotAreaValue = NaN
df_forSale_trunc.dropna(subset=['lotAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['lotAreaValue'], inplace=True)

# Reset any zestimate value from 0 to the price
indexList = df_forSale_trunc.index[df_forSale_trunc['zestimate'].isna()].tolist()
df_forSale_trunc.loc[indexList, 'zestimate'] = df_forSale_trunc.loc[indexList, 'price']
indexList = df_recentlySold_trunc.index[df_recentlySold_trunc['zestimate'].isna()].tolist()
df_recentlySold_trunc.loc[indexList, 'zestimate'] = df_recentlySold_trunc.loc[indexList, 'price']

# Drop all labels with lat/long = NaN
df_forSale_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)
df_recentlySold_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)

# Fix any missing or NaN cityId values
df_forSale_trunc.dropna(subset=['cityId'], inplace=True)
df_recentlySold_trunc.dropna(subset=['cityId'], inplace=True)
valueRecentlySold = {'replace_value':list(df_forSale_trunc['cityId'].mode())[-1]}
valueForSale = {'replace_value':list(df_forSale_trunc['cityId'].mode())[-1]}
df_recentlySold_trunc['cityId'] = cleanValue(df_recentlySold_trunc['cityId'].copy(), valueRecentlySold)
df_forSale_trunc['cityId'] = cleanValue(df_forSale_trunc['cityId'].copy(), valueForSale)

value = {'replace_value':0}
df_forSale_trunc['resoFacts.taxAnnualAmount'] = cleanValue(df_forSale_trunc['resoFacts.taxAnnualAmount'].copy(), value)
df_recentlySold_trunc['resoFacts.taxAnnualAmount'] = cleanValue(df_recentlySold_trunc['resoFacts.taxAnnualAmount'].copy(), value)
df_forSale_trunc['resoFacts.garageSpaces'] = cleanValue(df_forSale_trunc['resoFacts.garageSpaces'].copy(), value)
df_recentlySold_trunc['resoFacts.garageSpaces'] = cleanValue(df_recentlySold_trunc['resoFacts.garageSpaces'].copy(), value)

# Set cityId to string value
df_forSale_trunc['cityId'] = df_forSale_trunc['cityId'].astype(str)
df_recentlySold_trunc['cityId'] = df_recentlySold_trunc['cityId'].astype(str)

In [102]:
'''
Remove '.0' from key integer values
'''

for col in ['zipcode', 'cityId']:
    df_forSale_trunc[col] = df_forSale_trunc[col].apply(lambda x: x.split('.')[0])
    df_recentlySold_trunc[col] = df_recentlySold_trunc[col].apply(lambda x: x.split('.')[0])


In [103]:
'''
Change boolean columns to 1/0
'''

bool_features = ['resoFacts.isSeniorCommunity','resoFacts.hasWaterfrontView']

df_forSale_trunc = MultiColumnLabelEncoder(columns=bool_features).fit_transform(df_forSale_trunc.copy())
df_recentlySold_trunc = MultiColumnLabelEncoder(columns=bool_features).fit_transform(df_recentlySold_trunc.copy())

In [104]:
'''
Create new features.
'''

df_forSale_trunc['price/sqft'] = df_forSale_trunc['price'].astype(float)/df_forSale_trunc['livingAreaValue'].astype(float)
df_recentlySold_trunc['price/sqft'] = df_recentlySold_trunc['price'].astype(float)/df_recentlySold_trunc['livingAreaValue'].astype(float)

In [105]:
'''
One-Hot-Encode string numeric features.

Features to be One-Hot-Encoded: zipcode, yearBuilt, cityId, resoFacts.propertySubType	
'''

OHE_features = ['zipcode', 'yearBuilt', 'cityId']
zip_fs = df_forSale_trunc['zipcode'].copy()
zip_rs = df_recentlySold_trunc['zipcode'].copy()
yearBuilt_fs = df_forSale_trunc['yearBuilt'].copy()
yearBuilt_rs = df_recentlySold_trunc['yearBuilt'].copy()
df_forSale_trunc = pd.get_dummies(df_forSale_trunc, columns=OHE_features, dtype=int)
df_recentlySold_trunc = pd.get_dummies(df_recentlySold_trunc, columns=OHE_features, dtype=int)

df_forSale_trunc = removeDissimilarCols(df_forSale_trunc.copy(), df_recentlySold_trunc.copy(), 1)

dateString = df_recentlySold_trunc['dateSoldString'].copy()
df_recentlySold_trunc = removeDissimilarCols(df_recentlySold_trunc.copy(), df_forSale_trunc.copy(), 1)
df_recentlySold_trunc.insert(loc=0, column='dateSoldString', value=dateString)
df_forSale_trunc.insert(loc=1, column='zipcode', value=zip_fs)
df_recentlySold_trunc.insert(loc=1, column='zipcode', value=zip_rs)
df_forSale_trunc.insert(loc=2, column='yearBuilt', value=yearBuilt_fs)
df_recentlySold_trunc.insert(loc=2, column='yearBuilt', value=yearBuilt_rs)

In [106]:
'''
Update names and reset index.
'''

# Reset Index
df_forSale_trunc.reset_index(inplace=True)
df_forSale_trunc.drop(columns=['index'], inplace=True)

df_recentlySold_trunc.reset_index(inplace=True)
df_recentlySold_trunc.drop(columns=['index'], inplace=True)

# Find and create/update outlier feature

In [107]:
'''
Apply domain knowledge to remove any additioanl outliers.

- Shouldn't have any homes where there are more than 3 more bathrooms than bedrooms
- Shouldn't have any homes where sqft/bedroom < domain value
'''

df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['lotAreaValue']<=300000.0]
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['lotAreaValue']<=300000.0]

df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['livingAreaValue']<=7000.0] # Get this to be a lower<=x<upper type range
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['livingAreaValue']<=7000.0]# Get this to be a lower<=x<upper type range

df_recentlySold_trunc.reset_index(inplace=True, drop=True)
df_forSale_trunc.reset_index(inplace=True, drop=True)

In [108]:
'''
Check for outliers
'''

# Checking for outliers in list price and sold price based on zipcode
df_recentlySold_trunc.sort_values(by='price', inplace=True)
soldPrice = df_recentlySold_trunc[['zipcode','price']]
outlierBoolean_rs = outliers_Z_Score(sortedList=soldPrice, outlierBoolean=outlierBoolean_rs, outlierColumn='price', groupbyColumn='zipcode')
df_recentlySold_trunc.sort_index(inplace=True)

df_forSale_trunc.sort_values(by='price', inplace=True)
salePrice = df_forSale_trunc[['zipcode','price']]
outlierBoolean_fs = outliers_Z_Score(sortedList=salePrice, outlierBoolean=outlierBoolean_fs, outlierColumn='price', groupbyColumn='zipcode')
df_forSale_trunc.sort_index(inplace=True)

# Checking for outliers in price/sqft based on zipcode
df_recentlySold_trunc.sort_values(by='price/sqft', inplace=True)
soldPrice = df_recentlySold_trunc[['zipcode','price/sqft']]
outlierBoolean_rs = outliers_Z_Score(sortedList=soldPrice, outlierBoolean=outlierBoolean_rs, outlierColumn='price/sqft', groupbyColumn='zipcode')
df_recentlySold_trunc.sort_index(inplace=True)

df_forSale_trunc.sort_values(by='price/sqft', inplace=True)
salePrice = df_forSale_trunc[['zipcode','price/sqft']]
outlierBoolean_fs = outliers_Z_Score(sortedList=salePrice, outlierBoolean=outlierBoolean_fs, outlierColumn='price/sqft', groupbyColumn='zipcode')
df_forSale_trunc.sort_index(inplace=True)

# Checking for outliers in lotAreaValue
df_recentlySold_trunc.sort_values(by='lotAreaValue', inplace=True)
soldPrice = df_recentlySold_trunc.copy()
outlierBoolean_rs = outliersAll_Z_Score(sortedList=soldPrice, outlierBoolean=outlierBoolean_rs, outlierColumn='lotAreaValue')
df_recentlySold_trunc.sort_index(inplace=True)

df_forSale_trunc.sort_values(by='lotAreaValue', inplace=True)
salePrice = df_forSale_trunc.copy()
outlierBoolean_fs = outliersAll_Z_Score(sortedList=salePrice, outlierBoolean=outlierBoolean_fs, outlierColumn='lotAreaValue')
df_forSale_trunc.sort_index(inplace=True)

# Checking for outliers in livingAreaValue
df_recentlySold_trunc.sort_values(by='livingAreaValue', inplace=True)
soldPrice = df_recentlySold_trunc.copy()
outlierBoolean_rs = outliersAll_Z_Score(sortedList=soldPrice, outlierBoolean=outlierBoolean_rs, outlierColumn='livingAreaValue')
df_recentlySold_trunc.sort_index(inplace=True)

df_forSale_trunc.sort_values(by='livingAreaValue', inplace=True)
salePrice = df_forSale_trunc.copy()
outlierBoolean_fs = outliersAll_Z_Score(sortedList=salePrice, outlierBoolean=outlierBoolean_fs, outlierColumn='livingAreaValue')
df_forSale_trunc.sort_index(inplace=True)

# Checking for outliers in resoFacts.taxAnnualAmount
df_recentlySold_trunc.sort_values(by='resoFacts.taxAnnualAmount', inplace=True)
soldPrice = df_recentlySold_trunc.copy()
outlierBoolean_rs = outliersAll_Z_Score(sortedList=soldPrice, outlierBoolean=outlierBoolean_rs, outlierColumn='resoFacts.taxAnnualAmount')
df_recentlySold_trunc.sort_index(inplace=True)

df_forSale_trunc.sort_values(by='resoFacts.taxAnnualAmount', inplace=True)
salePrice = df_forSale_trunc.copy()
outlierBoolean_fs = outliersAll_Z_Score(sortedList=salePrice, outlierBoolean=outlierBoolean_fs, outlierColumn='resoFacts.taxAnnualAmount')
df_forSale_trunc.sort_index(inplace=True)


df_recentlySold_trunc.insert(3, 'Outlier', outlierBoolean_rs, True)
df_forSale_trunc.insert(1, 'Outlier', outlierBoolean_fs, True)

In [109]:
'''
Reorganize dataframes
'''
# Break columns up into useable pieces for later DF organization
colName_exception = ['zipcode', 'price', 'zpid', 'yearBuilt']
colName_unique = {
    'ForSale':list(set(df_forSale_trunc.columns) - set(df_recentlySold_trunc.columns)),
    'RecentlySold':list(set(df_recentlySold_trunc.columns) - set(df_forSale_trunc.columns))
}

colName_common = list(set(df_forSale_trunc.columns) - set(colName_unique.get('ForSale')))
colName_OHE = [ele for ele in colName_common if(ele.split('_')[0] in OHE_features)]
colName_common = list(set(colName_common) - set(colName_OHE))

colName_OHE = list(set(colName_OHE) - set(colName_exception))
colName_common = list(colName_common + colName_exception)


# Organize column names for df_forSale_trunc
colName_fs = [colName_exception[2], colName_exception[0], colName_exception[3]] 
tL = list(list(set(colName_unique.get('ForSale'))-set(colName_exception)) + list(set(colName_common) - set(colName_exception)))
for x in tL:
    colName_fs.append(x)
if OHE_on:
    colName_fs = colName_fs + sorted(colName_OHE)
colName_fs.append(colName_exception[1])
df_forSale_trunc = df_forSale_trunc[colName_fs]

# Organize column names for df_recentlySold_trunc
colName_rs = [colName_exception[2], colName_exception[0], colName_exception[3]] 
tL = list(list(set(colName_unique.get('RecentlySold'))-set(colName_exception)) + list(set(colName_common) - set(colName_exception)))
for x in tL:
    colName_rs.append(x)
if OHE_on:
    colName_rs = colName_rs + sorted(colName_OHE)
colName_rs.append(colName_exception[1])
df_recentlySold_trunc = df_recentlySold_trunc[colName_rs]


In [110]:
'''
Drop unneeded columns
'''

colDrop = ['hdpUrl', 'livingAreaUnits', 'lotAreaUnits']
df_forSale_trunc.drop(columns=colDrop, inplace=True)
df_recentlySold_trunc.drop(columns=colDrop, inplace=True)

In [111]:
'''
Convert date to datetime object
'''

df_recentlySold_trunc['dateSoldString'] = pd.to_datetime(df_recentlySold_trunc['dateSoldString'])
df_recentlySold_trunc.rename(columns={'dateSoldString':'dateSold'}, inplace=True)

# Check df.info and df.describe and duplicates

In [112]:
df_forSale_trunc.describe(include='all')

,zpid,zipcode,yearBuilt,resoFacts.taxAnnualAmount,lotAreaValue,price/sqft,zestimate,bathrooms,daysOnZillow,schoolMaxRating,resoFacts.isSeniorCommunity,bedrooms,livingAreaValue,schoolMinDistance,resoFacts.garageSpaces,longitude,Outlier,resoFacts.hasWaterfrontView,latitude,price
count,2.176000e+03,2176,2176.000000,2176.000000,2176.000000,2176.000000,2.176000e+03,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2.176000e+03
unique,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,32909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.396555e+08,NaN,1996.028033,3210.127757,15075.525478,259.408309,5.061812e+05,2.399816,70.198989,6.032169,0.026654,3.497243,1971.962776,2.015763,3.112592,-80.686823,0.053309,0.149816,28.171160,5.221207e+05
std,5.226711e+08,NaN,23.441699,10731.156428,22543.447143,117.708512,3.456293e+05,0.792540,83.801207,2.037098,0.161108,0.755055,704.111311,1.725358,26.110652,0.081036,0.224700,0.356973,0.221967,3.600996e+05
min,4.336783e+07,NaN,1901.000000,0.000000,0.000000,77.462492,8.130000e+04,1.000000,1.000000,2.000000,0.000000,0.000000,532.000000,0.100000,0.000000,-80.930824,0.000000,0.000000,27.833658,8.900000e+04
25%,4.345463e+07,NaN,1980.000000,759.250000,7840.800000,197.039161,3.250000e+05,2.000000,16.000000,4.000000,0.000000,3.000000,1539.000000,0.900000,2.000000,-80.728971,0.000000,0.000000,27.989104,3.349000e+05
50%,5.461246e+07,NaN,1999.000000,2459.000000,10018.800000,226.536367,3.877225e+05,2.000000,41.000000,6.000000,0.000000,3.000000,1858.000000,1.500000,2.000000,-80.679870,0.000000,0.000000,28.118417,3.982500e+05
75%,1.041336e+08,NaN,2021.000000,4316.000000,11761.200000,280.668490,5.713000e+05,3.000000,103.000000,8.000000,0.000000,4.000000,2246.500000,2.500000,2.000000,-80.639202,0.000000,0.000000,28.336720,5.942250e+05


In [113]:
df_recentlySold_trunc.describe(include='all')

,zpid,zipcode,yearBuilt,dateSold,resoFacts.taxAnnualAmount,lotAreaValue,price/sqft,zestimate,bathrooms,daysOnZillow,schoolMaxRating,resoFacts.isSeniorCommunity,bedrooms,livingAreaValue,schoolMinDistance,resoFacts.garageSpaces,longitude,Outlier,resoFacts.hasWaterfrontView,latitude,price
count,5.562000e+03,5562,5562.000000,5562,5562.000000,5562.000000,5562.000000,5.562000e+03,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5.562000e+03
unique,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,32940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.276031e+08,NaN,1995.345559,2023-10-07 02:31:11.844660224,2760.709996,14315.681001,236.977275,4.805852e+05,2.347735,127.718626,6.137900,0.033261,3.450198,1942.163430,1.957911,3.440129,-80.689711,0.056814,0.183387,28.187095,4.697087e+05
min,4.336795e+07,NaN,1901.000000,2023-02-15 00:00:00,0.000000,0.000000,0.158730,1.400000e+04,0.000000,1.000000,2.000000,0.000000,0.000000,504.000000,0.100000,0.000000,-80.928590,0.000000,0.000000,27.833752,2.200000e+02
25%,4.344590e+07,NaN,1979.000000,2023-08-21 00:00:00,819.250000,7405.200000,186.865269,3.104250e+05,2.000000,63.000000,4.000000,0.000000,3.000000,1490.000000,0.900000,1.000000,-80.733150,0.000000,0.000000,28.012023,3.029900e+05
50%,5.012488e+07,NaN,1997.000000,2023-10-19 00:00:00,2275.500000,10018.800000,218.192185,4.011500e+05,2.000000,117.000000,6.000000,0.000000,3.000000,1828.000000,1.500000,2.000000,-80.686338,0.000000,0.000000,28.162717,3.950000e+05
75%,1.041315e+08,NaN,2020.000000,2023-12-08 00:00:00,3891.000000,12196.000000,264.385231,5.251750e+05,3.000000,176.000000,8.000000,0.000000,4.000000,2240.750000,2.600000,2.000000,-80.640632,0.000000,0.000000,28.342644,5.150000e+05
max,2.134275e+09,NaN,2024.000000,2024-02-15 00:00:00,41224.000000,268765.200000,1055.099648,4.269500e+06,7.000000,364.000000,10.000000,1.000000,9.000000,6484.000000,13.700000,4040.000000,-80.457010,1.000000,1.000000,28.780830,4.250000e+06


In [114]:
df_forSale_trunc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2176 entries, 0 to 2175
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zpid                         2176 non-null   float64
 1   zipcode                      2176 non-null   object 
 2   yearBuilt                    2176 non-null   int64  
 3   resoFacts.taxAnnualAmount    2176 non-null   float64
 4   lotAreaValue                 2176 non-null   float64
 5   price/sqft                   2176 non-null   float64
 6   zestimate                    2176 non-null   float64
 7   bathrooms                    2176 non-null   float64
 8   daysOnZillow                 2176 non-null   float64
 9   schoolMaxRating              2176 non-null   float64
 10  resoFacts.isSeniorCommunity  2176 non-null   int64  
 11  bedrooms                     2176 non-null   float64
 12  livingAreaValue              2176 non-null   float64
 13  schoolMinDistance      

In [115]:
df_recentlySold_trunc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5562 entries, 0 to 5561
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   zpid                         5562 non-null   float64       
 1   zipcode                      5562 non-null   object        
 2   yearBuilt                    5562 non-null   int64         
 3   dateSold                     5562 non-null   datetime64[ns]
 4   resoFacts.taxAnnualAmount    5562 non-null   float64       
 5   lotAreaValue                 5562 non-null   float64       
 6   price/sqft                   5562 non-null   float64       
 7   zestimate                    5562 non-null   float64       
 8   bathrooms                    5562 non-null   float64       
 9   daysOnZillow                 5562 non-null   float64       
 10  schoolMaxRating              5562 non-null   float64       
 11  resoFacts.isSeniorCommunity  5562 non-null   int

In [116]:
for column_ in df_forSale_trunc.columns:
    nullValueCount = df_forSale_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in resoFacts.taxAnnualAmount: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in price/sqft: 0
Number of NaN values in zestimate: 0
Number of NaN values in bathrooms: 0
Number of NaN values in daysOnZillow: 0
Number of NaN values in schoolMaxRating: 0
Number of NaN values in resoFacts.isSeniorCommunity: 0
Number of NaN values in bedrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in schoolMinDistance: 0
Number of NaN values in resoFacts.garageSpaces: 0
Number of NaN values in longitude: 0
Number of NaN values in Outlier: 0
Number of NaN values in resoFacts.hasWaterfrontView: 0
Number of NaN values in latitude: 0
Number of NaN values in price: 0


In [117]:
for column_ in df_recentlySold_trunc.columns:
    nullValueCount = df_recentlySold_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in dateSold: 0
Number of NaN values in resoFacts.taxAnnualAmount: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in price/sqft: 0
Number of NaN values in zestimate: 0
Number of NaN values in bathrooms: 0
Number of NaN values in daysOnZillow: 0
Number of NaN values in schoolMaxRating: 0
Number of NaN values in resoFacts.isSeniorCommunity: 0
Number of NaN values in bedrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in schoolMinDistance: 0
Number of NaN values in resoFacts.garageSpaces: 0
Number of NaN values in longitude: 0
Number of NaN values in Outlier: 0
Number of NaN values in resoFacts.hasWaterfrontView: 0
Number of NaN values in latitude: 0
Number of NaN values in price: 0


In [118]:
# Check for duplicates
print(f'There are {df_forSale_trunc.price.loc[df_forSale_trunc.duplicated()].count()} duplicates in df_forSale_trunc.')
print(f'There are {df_recentlySold_trunc.price.loc[df_recentlySold_trunc.duplicated()].count()} duplicates in df_recentlySold_trunc.')

There are 0 duplicates in df_forSale_trunc.
There are 0 duplicates in df_recentlySold_trunc.


# Rescale recentlySold data to match distribution of forSale data -- If needed

In [119]:
'''
When updating the 'in' list below, there needs to be a better way to get the correct column names for later use...

Don't use this to start. Might need to later.
'''
# print(df_recentlySold_trunc['lotAreaValue'].max(), [df_recentlySold_trunc['lotAreaValue'][0:10], df_forSale_trunc['lotAreaValue'][0:10]])
# print('')
colName_fs = list(set(colName_fs) - set(colName_OHE) - set(['zipcode','yearBuilt','zpid','Outlier','hdpUrl','daysOnZillow','longitude','latitude','resoFacts.hasWaterfrontView','lotAreaUnits','livingAreaUnits','resoFacts.isSeniorCommunity']))

if removeOutliers:
    df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['Outlier']==0].copy()
    df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['Outlier']==0].copy()

if scale:
    print('Matching distributions now...')
    for i in df_forSale_trunc.columns:
        if i in colName_fs:
            df_recentlySold_trunc[i] = match_ecdf(df_recentlySold_trunc[i], df_forSale_trunc[i])

# print(df_recentlySold_trunc['lotAreaValue'].max(), [df_recentlySold_trunc['lotAreaValue'][0:10], df_forSale_trunc['lotAreaValue'][0:10]])

In [120]:
'''
Checking feature distribution similarities using two-sample Kolmogorov-Smirnov fit of goodness
'''

yes = 0
no = 0
diff_Feats = []
diff_pvalue = []

print('ATTENTION: Make sure the two dataframes have the same number of features and are in the same order!')
print('')

for i in colName_fs:
    # Assuming 'data1' and 'data2' are your datasets
    statistic, p_value = ks_2samp(df_forSale_trunc[i].values, df_recentlySold_trunc[i].values)
    if p_value < 0.05:
        # print(f'Feature {df_forSale.columns[i]} is significantly different.')
        diff_Feats.append(i)
        diff_pvalue.append(p_value)
        yes+=1
    else:
        # print(f'Feature {df_forSale.columns[i]} is NOT significantly different')
        no+=1

print(f'{yes} of {yes+no} ({round(yes/(yes+no)*100,2)}%) feature distributions are significantly different.')
if round(yes/(yes+no)*100,2) > 5:
    print('Recommend distribution matching!')
print('')
print(f'These are the features that differ:')
for i, feat in enumerate(diff_Feats):
    print(feat, diff_pvalue[i])

ATTENTION: Make sure the two dataframes have the same number of features and are in the same order!

9 of 11 (81.82%) feature distributions are significantly different.
Recommend distribution matching!

These are the features that differ:
livingAreaValue 0.038385635816756435
resoFacts.taxAnnualAmount 8.650482378019821e-05
resoFacts.garageSpaces 0.00037032861478897495
lotAreaValue 0.008136123409163667
price/sqft 9.302946096150108e-13
price 3.2595105667676605e-15
zestimate 5.5391489124566046e-06
schoolMaxRating 0.021965271382374016
bedrooms 0.043095951094787754


# Output final dataframes

In [121]:
'''
Create new CSV file with all cleaned and wrangled data.
'''

# Rename price column to be more clear
df_forSale_trunc.rename(columns={'price':'listPrice'}, inplace=True)
df_recentlySold_trunc.rename(columns={'price':'soldPrice'}, inplace=True)

# Export dataframes to csv
df_forSale_trunc.to_csv('cleaned_forSale_data.csv', sep=',', index=True, encoding='utf-8')
df_recentlySold_trunc.to_csv('cleaned_recentlySold_data.csv', sep=',', index=True, encoding='utf-8')
# del df_recentlySold_trunc, df_forSale_trunc

---
End of section.
## <pre><-- Previous: Data Collection                        Next: EDA --></pre>